In [ ]:
%load_ext watermark


In [ ]:
import itertools as it
import os

from IPython.display import display, HTML
import matplotlib as mpl
import polars as pl
from slugify import slugify
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
%watermark -diwmuv -iv


In [ ]:
tp.save[".pgf"] = True
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2025-06-30-cpp-bench-speed-tilted"
)
teeplot_subdir


## Prep Data


In [ ]:
algo_names = {
    "dstream.circular_algo": "simple ringbuf",
    "control_throwaway_algo": "no-operation",
    "dstream.tilted_algo": "extended ringbuf",
    "dstream_tilted_algo": "extended ringbuf LUT",
    "doubling_tilted_algo": "naive doubling",
    "zhao_tilted_algo": "pyramidal bucket",
    "zhao_tilted_full_algo": "saturating bucket",
}


df = pl.concat(
    [
        pl.read_csv("https://osf.io/cbqpx/download").with_columns(
            platform=pl.lit("ARM Cortex-M0+"),
        ),
        pl.read_csv("https://osf.io/ewmk9/download").with_columns(
            platform=pl.lit("ARM Apple M1"),
        ),
        pl.read_csv("https://osf.io/k3vbu/download").with_columns(
            platform=pl.lit("x86 Intel Core Ultra 7 155U"),
        ),
    ],
)
df = (
    df.cast(
        {
            "memory_bytes": pl.Int32,
            "num_items": pl.Int32,
            "num_sites": pl.Int32,
            "duration_s": pl.Float64,
            "replicate": pl.Int32,
        },
    )
    .with_columns(
        duration_per_item_ns=(
            pl.col("duration_s") * 1_000_000_000 / pl.col("num_items")
        )
    )
    .filter(
        pl.col("algo_name").is_in(
            algo_names.keys(),
        ),
    )
    .with_columns(
        algorithm=pl.col("algo_name").map_elements(
            algo_names.__getitem__,
            return_dtype=str,
        )
    )
    .with_columns(pl.col("data_type").alias("data type"))
)

display(df.describe()), display(df.head()), display(df.tail());


In [ ]:
platforms = df.select("platform").unique().to_series().to_list()
platforms


In [ ]:
hue_order = [
    "extended ringbuf",
    "extended ringbuf LUT",
    "saturating bucket",
    "naive doubling",
    "no-operation",
    "simple ringbuf",
    "pyramidal bucket",
]


In [ ]:
palette = [
    "#9C4897",
    "#6D61A7",
    "#63a28b",
    "#828282",
    "#fac746",
    "#89ccf2",
    "#baf23d",
]


## Plot


In [ ]:
for platform, rc, errorbar in it.product(
    platforms,
    [{}, {"font.family": "serif"}],
    [("ci", 95), "se"],
):
    with mpl.rc_context(rc=rc):
        display(HTML(f"<h2>{platform}</h2>"))
        with tp.teed(
            sns.relplot,
            data=df.filter(
                pl.col("platform") == platform,
            ).with_columns(
                pl.col("num_sites").alias("buffer size $S$"),
                pl.col("duration_per_item_ns").alias("per-item walltime (ns)"),
                pl.col("num_items").alias("num items ingested"),
            ),
            x="buffer size $S$",
            y="per-item walltime (ns)",
            col="num items ingested",
            row="data type",
            hue="algorithm",
            style="algorithm",
            palette=palette,
            errorbar=errorbar,
            kind="line",
            aspect=1.5,
            markers=["^", "X"],
            ms=8,
            height=2,
            teeplot_outattrs={
                "platform": slugify(platform),
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(margin_titles=True),
        ) as g:
            g.set(xscale="log", yscale="log")
            g.set_xlabels("Buffer Size $S$\n(item count)")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.35, 1),
                ncol=2,
                title=None,
                frameon=False,
            )
            g.tight_layout()


In [ ]:
for rc, errorbar in it.product(
    [{}, {"font.family": "serif"}],
    [("ci", 95), "se"],
):
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.relplot,
            data=df.filter(
                pl.col("num_items") == 100_000,
            ).with_columns(
                pl.col("num_sites").alias("buffer size $S$"),
                pl.col("duration_per_item_ns").alias("per-item\nwalltime (ns)"),
                pl.col("num_items").alias("num items ingested"),
                pl.col("platform").replace(
                    {
                        "ARM Cortex-M0+": "RP2040 Cortex-M0+",
                        "x86 Intel Core Ultra 7 155U": "Intel Core Ultra 7 155U",
                    },
                )
            ),
            x="buffer size $S$",
            y="per-item\nwalltime (ns)",
            col="data type",
            col_order=[
                "bit",
                # "byte",
                # "word",
                "double word",
            ],
            row="platform",
            row_order=[
                "Intel Core Ultra 7 155U",
                "RP2040 Cortex-M0+",
            ],
            hue="algorithm",
            hue_order=hue_order,
            style="algorithm",
            style_order=hue_order,
            palette=palette,
            kind="line",
            errorbar=errorbar,
            aspect=0.8,
            markers=["X", "X", "d", "d", ".", ".", "."],
            dashes=[(1, 0), (1, 0), (1, 0), (1, 0), (1, 4), (1, 4), (1, 4)],
            ms=8,
            height=2.8,
            teeplot_outattrs={
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(
                margin_titles=True,
                sharey="row",
            ),
        ) as g:
            g.set(xscale="log", yscale="log")
            g.set_titles(
                col_template="{col_name}\ndtype",
                row_template="{row_name}",
            )
            g.set_xlabels("Buffer Size $S$\n(item count)")
            sns.move_legend(
                g,
                "upper center",
                bbox_to_anchor=(0.34, 1.16),
                ncol=2,
                title=None,
                frameon=False,
            )
            for ax in g.axes.flat:
                ax.set_xticks([64, 256, 1024, 4096])
                ax.set_xticklabels(["64", "256", "1024", "4096"])
            g.tight_layout()


In [ ]:
for platform, rc, errorbar in it.product(
    platforms, [{}, {"font.family": "serif"}], [("ci", 95), "se"]
):
    display(HTML(f"<h2>{platform=}</h2>"))
    with mpl.rc_context(rc=rc):
        with tp.teed(
            sns.relplot,
            data=df.filter(
                pl.col("num_items") == 100_000,
                pl.col("platform") == platform,
            ).with_columns(
                pl.col("num_sites").alias("buffer size $S$"),
                pl.col("duration_per_item_ns").alias("per-item\nwalltime (ns)"),
                pl.col("num_items").alias("num items ingested"),
            ),
            x="buffer size $S$",
            y="per-item\nwalltime (ns)",
            col="data type",
            col_order=[
                "bit",
                "byte",
                "word",
                "double word",
            ],
            hue="algorithm",
            hue_order=hue_order,
            style="algorithm",
            style_order=hue_order,
            palette=palette,
            kind="line",
            aspect=0.7,
            errorbar=errorbar,
            markers=["^", "^", "P", "P", "*", "v", "X"],
            ms=8,
            height=2.8,
            teeplot_outattrs={
                "platform": slugify(platform),
                **rc,
            },
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(margin_titles=False),
        ) as g:
            g.set(xscale="log", yscale="log")
            g.set_titles(col_template="{col_name} dtype")
            g.set_xlabels("Buffer Size $S$\n(item count)")
            sns.move_legend(
                g,
                "upper center",
                bbox_to_anchor=(0.39, 1.16),
                ncol=4,
                title=None,
                frameon=False,
            )
            for ax in g.axes.flat:
                ax.set_xticks([64, 256, 1024, 4096])
                ax.set_xticklabels(["64", "256", "1024", "4096"])
            g.tight_layout()


In [ ]:
for platform, rc, errorbar in it.product(
    platforms, [{}, {"font.family": "serif"}], [("ci", 95), "se"]
):
    with mpl.rc_context(rc=rc):
        for num_sites in [64, 256, 1024, 4096]:
            display(HTML(f"<h2>{platform=} {num_sites=}</h2>"))
            with tp.teed(
                sns.relplot,
                data=df.filter(
                    pl.col("num_sites") == num_sites,
                    pl.col("platform") == platform,
                ).with_columns(
                    pl.col("num_sites").alias("buffer size $S$"),
                    pl.col("duration_per_item_ns").alias("per-item walltime (ns)"),
                    pl.col("num_items").alias("num items ingested"),
                ),
                x="num items ingested",
                y="per-item walltime (ns)",
                col="data type",
                col_order=[
                    "bit",
                    "byte",
                    "word",
                    "double word",
                ],
                hue="algorithm",
                hue_order=hue_order,
                style="algorithm",
                style_order=hue_order,
                palette=palette,
                kind="line",
                aspect=0.7,
                errorbar=errorbar,
                markers=["^", "^", "P", "P", "*", "v", "X"],
                ms=8,
                height=2.8,
                teeplot_outattrs={
                    "platform": slugify(platform),
                    **rc,
                },
                teeplot_subdir=teeplot_subdir,
                facet_kws=dict(margin_titles=False),
            ) as g:
                g.set(xscale="log", yscale="log")
                g.set_titles(col_template="{col_name} dtype")
                sns.move_legend(
                    g,
                    "upper center",
                    bbox_to_anchor=(0.39, 1.16),
                    ncol=4,
                    title=None,
                    frameon=False,
                )
                g.tight_layout()
